In [1]:
# import packages
import pandas as pd
import os
import sys
import mysql.connector
import numpy as np
import datetime

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor

import joblib
from joblib import load, dump
import pickle
import json

# Import dataset and Investigate

In [2]:
# import data in txt file
read_leavetimes = pd.read_csv ('/Users/danhowes1/Desktop/UCD/Research_Project/tmp/data/rt_leavetimes_DB_2018.txt', delimiter = ';')

In [3]:
# export data as csv
read_leavetimes.to_csv ('/Users/danhowes1/Desktop/UCD/Research_Project/rt_leavetimes_DB_2018.csv', index=None)

In [2]:
# import data as csv
df_leavetimes = pd.read_csv('/Users/danhowes1/Desktop/UCD/Research_Project/rt_leavetimes_DB_2018.csv')

In [3]:
# Check how many rows and columns this dataframe has
df_leavetimes.shape

(116949113, 18)

In [4]:
# show first 10 rows
df_leavetimes.head(10)

,DATASOURCE,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,PASSENGERS,PASSENGERSIN,PASSENGERSOUT,DISTANCE,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,01-JAN-18 00:00:00,5972116,12,119,48030,48030,48012,48012,2693211,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
1,DB,01-JAN-18 00:00:00,5966674,12,119,54001,54001,54023,54023,2693267,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
2,DB,01-JAN-18 00:00:00,5959105,12,119,60001,60001,59955,59955,2693263,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
3,DB,01-JAN-18 00:00:00,5966888,12,119,58801,58801,58771,58771,2693284,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
4,DB,01-JAN-18 00:00:00,5965960,12,119,56401,56401,56309,56323,2693209,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
5,DB,01-JAN-18 00:00:00,5965964,12,119,70688,70688,70663,70679,2693209,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
6,DB,01-JAN-18 00:00:00,5958117,12,119,72488,72488,72539,72539,2172293,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
7,DB,01-JAN-18 00:00:00,5959109,12,119,74288,74288,74173,74173,2693263,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
8,DB,01-JAN-18 00:00:00,5972114,12,119,40187,40187,40096,40111,2693211,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
9,DB,01-JAN-18 00:00:00,5959099,12,119,38387,38387,38271,38271,2693263,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN


In [5]:
#Now check type of each feature
df_leavetimes.dtypes

DATASOURCE          object
DAYOFSERVICE        object
TRIPID               int64
PROGRNUMBER          int64
STOPPOINTID          int64
PLANNEDTIME_ARR      int64
PLANNEDTIME_DEP      int64
ACTUALTIME_ARR       int64
ACTUALTIME_DEP       int64
VEHICLEID            int64
PASSENGERS         float64
PASSENGERSIN       float64
PASSENGERSOUT      float64
DISTANCE           float64
SUPPRESSED         float64
JUSTIFICATIONID    float64
LASTUPDATE          object
NOTE               float64
dtype: object

<b> - Look for duplicate rows and columns. Consider whether it makes sense to keep them or drop them. </b>

In [6]:
#Print the number of duplicates, without the original rows that were duplicated
print('Number of duplicate (excluding first) rows in the table is: ', df_leavetimes.duplicated().sum())

# Check for duplicate rows. 
# Use "keep=False" to mark all duplicates as true, including the original rows that were duplicated.
print('Number of duplicate rows (including first) in the table is:', df_leavetimes[df_leavetimes.duplicated(keep=False)].shape[0])


Number of duplicate (excluding first) rows in the table is:  0
Number of duplicate rows (including first) in the table is: 0


In [7]:
#check number of empty rows per feature
df_leavetimes.isnull().sum()

DATASOURCE                 0
DAYOFSERVICE               0
TRIPID                     0
PROGRNUMBER                0
STOPPOINTID                0
PLANNEDTIME_ARR            0
PLANNEDTIME_DEP            0
ACTUALTIME_ARR             0
ACTUALTIME_DEP             0
VEHICLEID                  0
PASSENGERS         116949113
PASSENGERSIN       116949113
PASSENGERSOUT      116949113
DISTANCE           116949113
SUPPRESSED         116360453
JUSTIFICATIONID    116360526
LASTUPDATE                 0
NOTE               116949113
dtype: int64

In [8]:
#check cardinality of each feature
df_leavetimes.nunique()

DATASOURCE              1
DAYOFSERVICE          360
TRIPID             658961
PROGRNUMBER           109
STOPPOINTID          4774
PLANNEDTIME_ARR     72712
PLANNEDTIME_DEP     72712
ACTUALTIME_ARR      74361
ACTUALTIME_DEP      74363
VEHICLEID            1151
PASSENGERS              0
PASSENGERSIN            0
PASSENGERSOUT           0
DISTANCE                0
SUPPRESSED              2
JUSTIFICATIONID     33487
LASTUPDATE            360
NOTE                    0
dtype: int64

# Cleaning Dataset

<b> - Drop constant columns </b>

In [9]:
constant_columns = ['DATASOURCE', 'PASSENGERS', 'PASSENGERSIN', 'PASSENGERSOUT', 'DISTANCE', 'NOTE', 'SUPPRESSED', 'JUSTIFICATIONID', 'LASTUPDATE']

for c in constant_columns:
    df_leavetimes = df_leavetimes.drop(c, 1)

df_leavetimes.head(10)

/var/folders/bn/6x11zktd6n14vfwyb0jymgt00000gp/T/ipykernel_44185/2347938632.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_leavetimes = df_leavetimes.drop(c, 1)


,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID
0,01-JAN-18 00:00:00,5972116,12,119,48030,48030,48012,48012,2693211
1,01-JAN-18 00:00:00,5966674,12,119,54001,54001,54023,54023,2693267
2,01-JAN-18 00:00:00,5959105,12,119,60001,60001,59955,59955,2693263
3,01-JAN-18 00:00:00,5966888,12,119,58801,58801,58771,58771,2693284
4,01-JAN-18 00:00:00,5965960,12,119,56401,56401,56309,56323,2693209
5,01-JAN-18 00:00:00,5965964,12,119,70688,70688,70663,70679,2693209
6,01-JAN-18 00:00:00,5958117,12,119,72488,72488,72539,72539,2172293
7,01-JAN-18 00:00:00,5959109,12,119,74288,74288,74173,74173,2693263
8,01-JAN-18 00:00:00,5972114,12,119,40187,40187,40096,40111,2693211
9,01-JAN-18 00:00:00,5959099,12,119,38387,38387,38271,38271,2693263


<b> - Planned arrival times vs Planned leave times for each stop </b>

In [10]:
df_leavetimes[df_leavetimes['PLANNEDTIME_ARR'] != df_leavetimes['PLANNEDTIME_DEP']]

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID


Clearly both columns contain the same information and we should drop one

In [11]:
df_leavetimes = df_leavetimes.drop('PLANNEDTIME_DEP', 1)


/var/folders/bn/6x11zktd6n14vfwyb0jymgt00000gp/T/ipykernel_44185/2842557364.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_leavetimes = df_leavetimes.drop('PLANNEDTIME_DEP', 1)


<b> - Actual arrival times vs Actual leave times for each stop </b>

In [12]:
df_leavetimes[df_leavetimes['ACTUALTIME_ARR'] != df_leavetimes['ACTUALTIME_DEP']]

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID
4,01-JAN-18 00:00:00,5965960,12,119,56401,56309,56323,2693209
5,01-JAN-18 00:00:00,5965964,12,119,70688,70663,70679,2693209
8,01-JAN-18 00:00:00,5972114,12,119,40187,40096,40111,2693211
10,01-JAN-18 00:00:00,5965956,12,119,41987,42017,42035,2693209
14,01-JAN-18 00:00:00,5972120,12,119,62378,62215,62238,2693211
...,...,...,...,...,...,...,...,...
116949102,31-DEC-18 00:00:00,8582940,61,670,59401,60339,60363,2534857
116949107,31-DEC-18 00:00:00,8587465,78,4383,65040,65656,65681,3265687
116949108,31-DEC-18 00:00:00,8588153,78,4383,28605,28998,29013,3265721
116949110,31-DEC-18 00:00:00,8586183,78,4383,51481,52237,52283,2693229


These two columns are different so we'll keep both

In [13]:
# check cardinality of new features
df_leavetimes.nunique()

DAYOFSERVICE          360
TRIPID             658961
PROGRNUMBER           109
STOPPOINTID          4774
PLANNEDTIME_ARR     72712
ACTUALTIME_ARR      74361
ACTUALTIME_DEP      74363
VEHICLEID            1151
dtype: int64

<b> - Logic check </b>

- Check departure times are greater than arrival times

In [14]:
df_leavetimes[df_leavetimes['ACTUALTIME_DEP'] < df_leavetimes['ACTUALTIME_ARR']]

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID
74759275,21-AUG-18 00:00:00,7321629,28,7348,45763,46208,46207,1932360
81013777,06-SEP-18 00:00:00,7643386,60,4848,36187,39260,39258,2406891
86217868,12-SEP-18 00:00:00,8088748,60,4848,70848,71991,71989,3265698


- Looks like an error in these rows, fix so they're the same value

In [15]:
rows = [74759275, 81013777, 86217868]
for i in rows:
    df_leavetimes.at[i, 'ACTUALTIME_DEP'] = df_leavetimes['ACTUALTIME_ARR'][i]
    
df_leavetimes[df_leavetimes['ACTUALTIME_DEP'] < df_leavetimes['ACTUALTIME_ARR']]

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID


In [16]:
df_leavetimes.head(10)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID
0,01-JAN-18 00:00:00,5972116,12,119,48030,48012,48012,2693211
1,01-JAN-18 00:00:00,5966674,12,119,54001,54023,54023,2693267
2,01-JAN-18 00:00:00,5959105,12,119,60001,59955,59955,2693263
3,01-JAN-18 00:00:00,5966888,12,119,58801,58771,58771,2693284
4,01-JAN-18 00:00:00,5965960,12,119,56401,56309,56323,2693209
5,01-JAN-18 00:00:00,5965964,12,119,70688,70663,70679,2693209
6,01-JAN-18 00:00:00,5958117,12,119,72488,72539,72539,2172293
7,01-JAN-18 00:00:00,5959109,12,119,74288,74173,74173,2693263
8,01-JAN-18 00:00:00,5972114,12,119,40187,40096,40111,2693211
9,01-JAN-18 00:00:00,5959099,12,119,38387,38271,38271,2693263


- Change name of columns so they dont clash with RT_Trips data 

In [17]:
df_leavetimes.rename(columns = {'PLANNEDTIME_ARR':'PLANNEDSTOPTIME_ARR/DEP', 'ACTUALTIME_ARR':'ACTUALSTOPTIME_ARR', 'ACTUALTIME_DEP':'ACTUALSTOPTIME_DEP'}, inplace = True)
df_leavetimes.head(5)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDSTOPTIME_ARR/DEP,ACTUALSTOPTIME_ARR,ACTUALSTOPTIME_DEP,VEHICLEID
0,01-JAN-18 00:00:00,5972116,12,119,48030,48012,48012,2693211
1,01-JAN-18 00:00:00,5966674,12,119,54001,54023,54023,2693267
2,01-JAN-18 00:00:00,5959105,12,119,60001,59955,59955,2693263
3,01-JAN-18 00:00:00,5966888,12,119,58801,58771,58771,2693284
4,01-JAN-18 00:00:00,5965960,12,119,56401,56309,56323,2693209


In [18]:
# save cleaned data as new csv
# df_leavetimes.to_csv ('/Users/danhowes1/Desktop/UCD/Research_Project/rt_leavetimes_DB_2018_CLEANED.csv', index=None)